In [31]:
# imstall / import required libraries

import os
import base64
import numpy as np
import tensorflow as tf

In [32]:
# load data

current_dir = os.getcwd()

data_path = os.path.join(current_dir, "mnist.npz")

(training_images, training_labels), (testing_images, testing_labels) = tf.keras.datasets.mnist.load_data(path=data_path)

print(training_images.shape, testing_images.shape)

(60000, 28, 28) (10000, 28, 28)


In [33]:
# normalise data

training_images = training_images.reshape(60000, 28, 28, 1)

training_images = training_images / 255

testing_images = testing_images.reshape(10000, 28, 28, 1)

testing_images = testing_images / 255

print(training_images.shape, testing_images.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [34]:
# callback to stop training early

class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):

        if logs['accuracy'] >= 0.995:

            self.model.stop_training = True

            print("\nReached > 0.995% accuracy thus stopping training.")

In [35]:
# create model

model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(28,28,1)),
    tf.keras.layers.Conv2D(128,(6,6), activation='relu'),
    tf.keras.layers.MaxPooling2D(4,4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [36]:
# train the model

model.fit(training_images, training_labels, epochs=50, callbacks=[EarlyStoppingCallback()])

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.8826 - loss: 0.4252
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9828 - loss: 0.0551
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9889 - loss: 0.0361
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9912 - loss: 0.0284
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9932 - loss: 0.0199
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9953 - loss: 0.0154
Epoch 7/50
1870/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9962 - loss: 0.0116
Reached > 0.995% accuracy thus stopping training.
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9962 - loss: 0.0117


In [37]:
predictions = model.predict(testing_images)
predictions = np.argmax(predictions,axis=1)
matches = (predictions == testing_labels)
match_count = np.sum(matches)
total_count = predictions.size
success = (match_count / total_count) * 100
print(success)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
99.13
